In [2]:
#train
import sys
from collections import defaultdict
counts=defaultdict(lambda:0)
context_counts=defaultdict(lambda:0)
with open("test02-train-input.txt","r") as training_file:
    for line in training_file:
        line=line.strip()
        words=line.split()
        words.append("</s>")
        words.insert(0,"<s>")
        for num in range(1,len(words)-1):
            counts[words[num-1]+" "+words[num]]+=1
            context_counts[words[num-1]]+=1
            counts[words[num]]+=1
            context_counts[""]+=1
    
with open("test02-train-answer.txt","w") as file:
    for ngram,count in counts.items():
        context=ngram.split(" ")
        if len(context)>=2:
            context=context.pop(0)
        else:
            context="" #unigram
        probability=float(counts[ngram]/context_counts[context])
        #print(ngram,probability)
        file.write("{}\t{}\n".format(ngram,probability))
        
#test
import math
lam1=0.95
lam2=0.60
V=1000000
W=0
H=0

probs=defaultdict(lambda:0)
with open("test02-train-answer.txt","r") as model_file:
    for line in model_file:
        data=line.split("\t")
        probs[data[0]]=float(data[1])
        
with open("datawiki-en-test.txt","r") as test_file:       
    for line in test_file:
        words=line.split()
        words.append("</s>")
        words.insert(0,"<s>")
        for num in range(1,len(words)-1):
            P1=lam1*probs[words[num]]+(1-lam1)/V
            P2=lam2*probs[words[num-1]+" "+words[num]]+(1-lam2)*P1
            H+=-math.log(P2,2)
            W+=1
    print(f"entropy={H/W}")

entropy=3.2517792022213396
